In [ ]:
%pylab

In [ ]:
%matplotlib inline

In [ ]:
import os
os.chdir('..')

In [ ]:
import GPy, scipy
from phenom.design import Formula
from phenom.phenotype import Phenotype
import pandas as pd

In [ ]:
n = 20
r = 3

x = np.linspace(-1, 1, n)[:,None]

k1 = GPy.kern.RBF(1)
k2 = GPy.kern.RBF(1, variance=.2)

mu = scipy.stats.multivariate_normal.rvs(np.zeros(n), k1.K(x))
b = scipy.stats.multivariate_normal.rvs(np.zeros(n), k2.K(x), size=r)

d = np.zeros((r, r+1))
d[:,0] = 1
d[:,1:] = np.eye(r)

yhat = np.dot(d, np.concatenate((mu[:,None], b.T), 1).T)
y = yhat + scipy.stats.norm.rvs(scale=.1, size=yhat.shape)

plt.plot(x[:,0], yhat.T)
plt.plot(x, mu, c='k')

In [ ]:
plt.plot(x[:,0], y.T)
plt.plot(x, mu, c='k')

In [ ]:
meta = pd.DataFrame(np.where(d[:,1:]!=0)[1], columns=['batch'])
data = pd.DataFrame(y.T, index=x[:,0])

mnull = Formula(meta, '1')
mfull = mnull + Formula(meta, 'C(batch) + 0')

In [ ]:
mn = Phenotype(data, mnull, maxExpectedCross=4, minExpectedCross=.1)
mf = Phenotype(data, mfull, maxExpectedCross=4, minExpectedCross=.1)

In [ ]:
nsamples = mn.samples()
fsamples = mf.samples()

In [ ]:
nsamples

In [ ]:
fsamples

In [ ]:
nsamp = nsamples.extract()
fsamp = fsamples.extract()

In [ ]:
ind = 0
m = nsamp['f'][:,ind,:].mean(0)
s = nsamp['f'][:,ind,:].std(0)

plt.plot(x, m)
plt.fill_between(x[:,0], m-2*s, m+2*s, alpha=.4)

m = fsamp['f'][:,ind,:].mean(0)
s = fsamp['f'][:,ind,:].std(0)

plt.plot(x, m)
plt.fill_between(x[:,0], m-2*s, m+2*s, alpha=.4)

plt.plot(x, mu, c='k')